# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
import emissor
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis

/Users/piek/Desktop/t-MA-Combots-2021/code/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os
import time
import uuid
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
from datetime import datetime
from datetime import date
import requests

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util

In [3]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
agent = "Leolani2"
human = "Stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, agent)

Directory  ./data/2021-10-28-17:28:25  Created 
Directory  ./data/2021-10-28-17:28:25/image  Created 


In [4]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-10-28 17:28:28,210 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-10-28 17:28:28,211 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain
2021-10-28 17:28:30,098 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain
2021-10-28 17:28:30,102 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-10-28 17:28:30,587 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-10-28 17:28:32,800 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-10-28 17:28:32,802 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-10-28 17:28:32,804 -    DEBUG -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-10-28 17:28:32,806 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-10-28 17:28:33,098 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query
2021-10-28 17:28:33,103 -     INF

In [13]:
def process_text_and_think (scenario: Scenario, 
                  place_id:str, 
                  location: str, 
                  textSignal: TextSignal,
                  human: str,
                  my_brain:LongTermMemory):
    thoughts = ""
    chat = Chat(human)
    chat.add_utterance([UtteranceHypothesis(c_util.seq_to_text(textSignal.seq), 1.0)])
    chat.last_utterance.analyze()
    # No triple was extracted, so we missed three items (s, p, o)
    if chat.last_utterance.triple is None:
        utterance = "Any gossip" + '\n'
    else:
        triple = c_util.rephrase_triple_json_for_capsule(chat.last_utterance.triple)
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_utterance_and_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal, 
                                                                  human,
                                                                  chat.last_utterance.perspective, 
                                                                  triple)
        #print('Capsule:', capsule)
        thoughts = my_brain.update(capsule, reason_types=True)
        #print(thoughts)
    return thoughts    

In [ ]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + human + "?"
print(agent + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(human + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)
    thoughts = process_text_and_think (scenario, 
                  place_id, 
                  location, 
                  textSignal, 
                  human, my_brain)
    
    # Create the response from the system and store this as a new signal
    utterance = "" #ttt.getTextFromTriples(response)
    if not utterance:
            utterance = "So you what do you want to talk about?\n"

    response = utterance[::-1]
    print(agent + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')


Leolani2: Hi there. Who are you Stranger?



 I am Piek


Stranger: I am Piek
2021-10-28 17:32:59,081 -     INFO - cltl.triple_extraction.api.Chat (Stranger)          000 - << Start of Chat with Stranger >>
2021-10-28 17:32:59,082 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-10-28 17:32:59,084 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-10-28 17:32:59,091 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-10-28 17:33:00,244 -     INFO - cltl.triple_extraction.api.Chat (Stranger)          001 -   Stranger: "I am Piek"
{'subject': {'label': 'Stranger', 'type': ['noun.person']}, 'predicate': {'label': 'be', 'type': ['verb.stative']}, 'object': {'label': 'Piek', 'type': ['person']}}
{'subject': {'Stranger', 'person'}, 'predicate': {'stative', 'be'}, 'object': {'Piek', 'person'}}
2021-10-28 17:33:01,621 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-10-28 17:33:01,659 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Po